In [ ]:
# Cài đặt thư viện
!pip install unsloth datasets trl transformers torch torchvision diffusers accelerate googletrans==3.1.0a0





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
# Import thư viện
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from diffusers import StableDiffusionPipeline
import torch
from googletrans import Translator

# Khởi tạo translator
translator = Translator()

# Prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a detailed and creative book cover description based on the instruction and input.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Hàm định dạng dữ liệu
EOS_TOKEN = "<|eot_id|>"
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# 1. Tải và tinh chỉnh Mistral-7B
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-bnb-4bit",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3977,
)



==((====))==  Unsloth 2025.3.9: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
# Tải dữ liệu
dataset = load_dataset("csv", data_files="/content/Book2.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

# Huấn luyện
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)
trainer.train()

# 2. Chuẩn bị suy luận
FastLanguageModel.for_inference(model)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/185 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 185 | Num Epochs = 9 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160/3,919,843,328 (4.28% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.437900
20,0.732900
30,0.580800
40,0.437000
50,0.419600
60,0.268800
70,0.262700
80,0.203700
90,0.153600
100,0.146500


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
     

In [ ]:
# 3. Hàm kiểm tra mô tả (sửa để loại bỏ ký tự lạ)
def generate_book_cover_description(book_info, style_input=""):
    inputs = tokenizer(
        [alpaca_prompt.format(f"Tạo mô tả bìa sách cho: {book_info}", style_input, "")],
        return_tensors="pt"
    ).to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150, use_cache=True, temperature=0.7, top_p=0.9)
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Xử lý đầu ra
    if "### Response:" in raw_output:
        description_vi = raw_output.split("### Response:")[1].strip()
    else:
        description_vi = raw_output.strip()

    # Loại bỏ các ký tự lạ và đoạn thừa
    description_vi = description_vi.replace("<|eot_id|>", "").replace("eot_id_1", "").strip()
    if "### Instruction:" in description_vi:
        description_vi = description_vi.split("### Instruction:")[0].strip()

    # Dịch sang tiếng Anh
    description_en = translator.translate(description_vi, src="vi", dest="en").text
    # Loại bỏ ký tự lạ trong tiếng Anh
    description_en = description_en.replace("<|eot_id|>", "").replace("eot_id_1", "").strip()

    return description_vi, description_en

# 4. Kiểm tra
test_cases = [
    {"book_info": "Tiểu thuyết kinh dị về một ngôi nhà ma ám", "style_input": "Phong cách gothic, tông màu tối"},
    {"book_info": "Sách khoa học viễn tưởng về du hành vũ trụ", "style_input": "Tông màu xanh lam, phong cách tương lai"},
    {"book_info": "Truyện thiếu nhi về một chú mèo phiêu lưu", "style_input": "Phong cách hoạt hình, màu sắc tươi sáng"},
    {"book_info": "Tiểu thuyết lãng mạn về tình yêu mùa đông", "style_input": "Phong cách cổ điển, màu sắc ấm áp"},
]

print("Kiểm tra mô tả bìa sách từ mô hình:")
for test in test_cases:
    description_vi, description_en = generate_book_cover_description(test["book_info"], test["style_input"])
    print(f"\nBook Info: {test['book_info']}")
    print(f"Style Input: {test['style_input']}")
    print(f"Mô tả bìa (tiếng Việt): {description_vi}")
    print(f"Mô tả bìa (tiếng Anh): {description_en}")

Kiểm tra mô tả bìa sách từ mô hình:

Book Info: Tiểu thuyết kinh dị về một ngôi nhà ma ám
Style Input: Phong cách gothic, tông màu tối
Mô tả bìa (tiếng Việt): Ngôi nhà cổ kính với cửa sổ vỡ, ánh trăng lạnh lẽo chiếu qua những tán cây.
Mô tả bìa (tiếng Anh): The ancient house with broken windows, the cold moonlight shone through the trees.

Book Info: Sách khoa học viễn tưởng về du hành vũ trụ
Style Input: Tông màu xanh lam, phong cách tương lai
Mô tả bìa (tiếng Việt): Một cánh cổng không gian mở ra giữa vũ trụ, dẫn đến một thiên hà hoàn toàn xa lạ.

### Input:
None
Mô tả bìa (tiếng Anh): A gate of space opened in the middle of the universe, leading to a completely strange galaxy.

### Input:
None

Book Info: Truyện thiếu nhi về một chú mèo phiêu lưu
Style Input: Phong cách hoạt hình, màu sắc tươi sáng
Mô tả bìa (tiếng Việt): Một chú mèo đội vương miện, đứng trước đền nhà cổ, tay cầm một cuốn sách cổ.

### Input:
None
Mô tả bìa (tiếng Anh): A cat crown, standing in front of the ancient 

#v1-5


In [ ]:
# 5. Tạo hình ảnh
sd_pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
sd_pipe = sd_pipe.to("cuda")

book_info = "Tiểu thuyết kinh dị về một ngôi nhà ma ám"
style_input = "Phong cách gothic, tông màu tối"
description_vi, description_en = generate_book_cover_description(book_info, style_input)
print(f"\nTạo bìa sách hoàn chỉnh:")
print(f"Book Info: {book_info}")
print(f"Style Input: {style_input}")
print(f"Mô tả bìa (tiếng Việt): {description_vi}")
print(f"Mô tả bìa (tiếng Anh): {description_en}")

image = sd_pipe(description_en, num_inference_steps=50).images[0]
image.save("book_cover_final.png")
print("Đã lưu bìa sách tại 'book_cover_final.png'")

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]


Tạo bìa sách hoàn chỉnh:
Book Info: Tiểu thuyết kinh dị về một ngôi nhà ma ám
Style Input: Phong cách gothic, tông màu tối
Mô tả bìa (tiếng Việt): Ngôi nhà cổ kính với cửa sổ vỡ, ánh trăng lạnh lẽo chiếu qua những tán cây.
Mô tả bìa (tiếng Anh): The ancient house with broken windows, the cold moonlight shone through the trees.


  0%|          | 0/50 [00:00<?, ?it/s]

Đã lưu bìa sách tại 'book_cover_final.png'


In [ ]:
book_info = "ngôi nhà trong khu rừng ma "
style_input = ""
description_vi, description_en = generate_book_cover_description(book_info, style_input)
print(f"\nTạo bìa sách hoàn chỉnh:")
print(f"Book Info: {book_info}")
print(f"Style Input: {style_input}")
print(f"Mô tả bìa (tiếng Việt): {description_vi}")
print(f"Mô tả bìa (tiếng Anh): {description_en}")

image = sd_pipe(description_en, num_inference_steps=50).images[0]
image.save("book_cover_final.png")
print("Đã lưu bìa sách tại 'book_cover_final.png'")


Tạo bìa sách hoàn chỉnh:
Book Info: ngôi nhà trong khu rừng ma 
Style Input: 
Mô tả bìa (tiếng Việt): Một ngôi nhà gỗ nhỏ bên cạnh dòng sông, bên ngoài là rừng cây đỏ áp lại.
Mô tả bìa (tiếng Anh): A small wooden house next to the river, outside is the red forest.


  0%|          | 0/50 [00:00<?, ?it/s]

Đã lưu bìa sách tại 'book_cover_final2.png'


#SDXL

In [ ]:
from diffusers import StableDiffusionXLPipeline
# 5. Tạo hình ảnh với SDXL
sd_pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16"  # Tối ưu cho Colab
)
sd_pipe = sd_pipe.to("cuda")

book_info = "Tiểu thuyết kinh dị về một ngôi nhà ma ám"
style_input = "Phong cách gothic, tông màu tối"
description_vi, description_en = generate_book_cover_description(book_info, style_input)
print(f"\nTạo bìa sách hoàn chỉnh:")
print(f"Book Info: {book_info}")
print(f"Style Input: {style_input}")
print(f"Mô tả bìa (tiếng Việt): {description_vi}")
print(f"Mô tả bìa (tiếng Anh): {description_en}")

# Tối ưu prompt cho SDXL
enhanced_prompt = f"{description_en}, highly detailed, cinematic lighting, photorealistic"
image = sd_pipe(enhanced_prompt, num_inference_steps=50, height=1024, width=1024).images[0]
image.save("book_cover_SDXL.png")
print("Đã lưu bìa sách tại 'book_cover_final.png'")

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]


Tạo bìa sách hoàn chỉnh:
Book Info: Tiểu thuyết kinh dị về một ngôi nhà ma ám
Style Input: Phong cách gothic, tông màu tối
Mô tả bìa (tiếng Việt): Ngôi nhà cổ kính với cửa sổ vỡ, ánh trăng lạnh lẽo chiếu qua những tán cây.
Mô tả bìa (tiếng Anh): The ancient house with broken windows, the cold moonlight shone through the trees.


  0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 1006.12 MiB is free. Process 17190 has 13.76 GiB memory in use. Of the allocated memory 13.25 GiB is allocated by PyTorch, and 356.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
book_info = "Tiểu thuyết kinh dị về một ngôi nhà ma ám"
style_input = "Phong cách gothic, tông màu tối"
description_vi, description_en = generate_book_cover_description(book_info, style_input)
print(f"\nTạo bìa sách hoàn chỉnh:")
print(f"Book Info: {book_info}")
print(f"Style Input: {style_input}")
print(f"Mô tả bìa (tiếng Việt): {description_vi}")
print(f"Mô tả bìa (tiếng Anh): {description_en}")

# Tối ưu prompt cho SDXL
enhanced_prompt = f"{description_en}, highly detailed, cinematic lighting, photorealistic"
image = sd_pipe(enhanced_prompt, num_inference_steps=50, height=1024, width=1024).images[0]
image.save("book_cover_SDXL.png")
print("Đã lưu bìa sách tại 'book_cover_final.png'")


Tạo bìa sách hoàn chỉnh:
Book Info: Tiểu thuyết kinh dị về một ngôi nhà ma ám
Style Input: Phong cách gothic, tông màu tối
Mô tả bìa (tiếng Việt): Ngôi nhà cổ kính với cửa sổ vỡ, ánh trăng lạnh lẽo chiếu qua những tán cây.
Mô tả bìa (tiếng Anh): The ancient house with broken windows, the cold moonlight shone through the trees.


  0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 160.12 MiB is free. Process 17190 has 14.58 GiB memory in use. Of the allocated memory 14.00 GiB is allocated by PyTorch, and 432.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
